# This notebook is trying to get a NER/TokenClassification dataset ready for Cleanlab

It's so confusing and hard to explain

- the debugging is mainly to do with getting the right shapes and types
- basically lots of converting from Torch to NumPy
- also need to drop the -100 labels from tokenizer
- hard to understand whether we are supposed to be dropping the IOB tags or not
- in the docs, which uses 3rd party libraries to make things more confusing, it seems they DROP the IOB stuff:

[https://github.com/cleanlab/examples/blob/master/entity_recognition/entity_recognition_training.ipynb](https://github.com/cleanlab/examples/blob/master/entity_recognition/entity_recognition_training.ipynb)

so reread the above maybe, I don't see where the step is that merges the labels

---

# Model/dataset stuff notes

Basically you need to generate what they are calling "out of sample predictions" : this just means generate probabilities/logits for each example **USING A MODEL THAT HAS NOT PREVIOUSLY SEEN THAT EXAMPLE DURING TRAINING**

1. Can do e.g. a 5x folds on 80/20 rolling basis across your dataset to get 5 separate models, and the concat the predictions over your entire dataset
2. **WHAT I DID WHILE STARTING OUT:** I have a finetuned model that was trained on the **TRAIN** split of Conll2003. So here I computed the "out of sample predictions" on the **TEST SPLIT** which means of course that I would only be able to "fix annotations" on this test split - **it was just a proof of principle, in reality want to do this on entire dataset hence the approach 1 above is needed**

In [1]:
from huggingface_hub import login
login()

In [21]:
from datasets import load_dataset

#wnut = load_dataset("wnut_17")
conll = load_dataset("eriktks/conll2003")

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1491: FutureWarning: The repository for eriktks/conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/eriktks/conll2003
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [121]:
label_list = conll["train"].features[f"ner_tags"].feature.names

label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("benjaminzwhite/finetuned-distilbert-base-uncased-conll2003-test-after-Optuna-HP-search")

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [6]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [23]:
tokenized_conll = conll.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [24]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [25]:
import numpy as np



In [26]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("benjaminzwhite/finetuned-distilbert-base-uncased-conll2003-test-after-Optuna-HP-search")


In [27]:
test_text = "I live in New York and my name is Frank Smith"
inputs = tokenizer(test_text, return_tensors="pt")

In [28]:
inputs

{'input_ids': tensor([[ 101, 1045, 2444, 1999, 2047, 2259, 1998, 2026, 2171, 2003, 3581, 3044,
          102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [48]:
print(type(inputs["input_ids"]), inputs["input_ids"].shape)

<class 'torch.Tensor'> torch.Size([1, 13])


In [15]:
import torch

In [44]:
with torch.no_grad():
    logits = model(**inputs).logits

In [17]:
logits

tensor([[[ 4.0091e+00, -4.1987e-01, -1.2509e+00, -4.5310e-01, -1.1621e+00,
          -7.5494e-01, -1.4429e+00, -2.4016e-01, -1.4042e+00],
         [ 7.5037e+00, -4.7691e-01, -2.1188e+00, -9.0407e-01, -1.4751e+00,
          -1.2813e+00, -2.3730e+00, -9.1726e-01, -1.8580e+00],
         [ 7.3623e+00, -9.9123e-01, -2.7489e+00, -1.0559e+00, -1.0318e+00,
          -5.6590e-01, -2.1219e+00, -7.5454e-01, -1.4494e+00],
         [ 7.4979e+00, -1.2394e+00, -2.3116e+00, -1.6666e+00, -1.5106e+00,
           4.1257e-02, -1.1240e+00, -8.5659e-01, -1.6617e+00],
         [-5.7956e-01, -8.5593e-01, -1.5546e+00, -1.7025e-01, -1.5773e+00,
           7.1416e+00,  8.1955e-01, -6.4093e-01, -1.9439e+00],
         [-8.9594e-01, -9.3039e-01,  2.1070e-01, -2.0779e+00, -5.4414e-01,
           3.1859e-01,  7.2289e+00, -1.6724e+00,  4.5929e-01],
         [ 8.2542e+00, -1.0705e+00, -1.6970e+00, -1.6428e+00, -8.9763e-01,
          -1.5288e+00, -1.5086e+00, -1.6676e+00, -1.2826e+00],
         [ 8.1564e+00, -6.3580e-01

In [18]:
logits.shape

torch.Size([1, 13, 9])

In [29]:
test_pred_probs = torch.nn.functional.softmax(logits, dim=-1) # CHECK THIS IS THE RIGHT DIM ???

test_pred_probs

tensor([[[9.3819e-01, 1.1190e-02, 4.8741e-03, 1.0824e-02, 5.3266e-03,
          8.0038e-03, 4.0227e-03, 1.3392e-02, 4.1813e-03],
         [9.9873e-01, 3.4162e-04, 6.6144e-05, 2.2286e-04, 1.2591e-04,
          1.5283e-04, 5.1295e-05, 2.1993e-04, 8.5849e-05],
         [9.9840e-01, 2.3519e-04, 4.0559e-05, 2.2047e-04, 2.2585e-04,
          3.5987e-04, 7.5927e-05, 2.9800e-04, 1.4874e-04],
         [9.9846e-01, 1.6023e-04, 5.4842e-05, 1.0453e-04, 1.2217e-04,
          5.7671e-04, 1.7985e-04, 2.3498e-04, 1.0505e-04],
         [4.4152e-04, 3.3491e-04, 1.6653e-04, 6.6483e-04, 1.6280e-04,
          9.9591e-01, 1.7888e-03, 4.1524e-04, 1.1283e-04],
         [2.9484e-04, 2.8486e-04, 8.9166e-04, 9.0421e-05, 4.1916e-04,
          9.9324e-04, 9.9575e-01, 1.3564e-04, 1.1433e-03],
         [9.9947e-01, 8.9148e-05, 4.7648e-05, 5.0300e-05, 1.0597e-04,
          5.6374e-05, 5.7522e-05, 4.9066e-05, 7.2108e-05],
         [9.9939e-01, 1.5182e-04, 3.7985e-05, 9.9523e-05, 8.2944e-05,
          6.0350e-05, 3.136

In [31]:
predictions = torch.argmax(logits, dim=2)
predictions


tensor([[0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 1, 2, 0]])

In [32]:
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
predicted_token_class

['O',
 'O',
 'O',
 'O',
 'B-LOC',
 'I-LOC',
 'O',
 'O',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'O']

# Trying to get model predictions to pred_probs across test dataset

In [35]:
tokenized_conll

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [109]:
# THIS IS WHERE I STORE THE INFO NEEDED FOR CLEANLAB
# PREFIX WITH X BECAUSE OF REUSE OF VARNAMES ITS SO CONFUSING
xtokens = []
xlabels = []
xpred_probs = []

cnt = 0
with torch.no_grad():
    for elem in tokenized_conll["test"]:
        #print(elem)
        # i need input ids and attention mask for the INPUTS variable
        # they need to be torch tensors also
        
        ii = torch.tensor(elem["input_ids"]).unsqueeze(dim=0)
        am = torch.tensor(elem["attention_mask"]).unsqueeze(dim=0)
        
        curr_inputs = {"input_ids":ii, "attention_mask":am}
        
        logits = model(**curr_inputs).logits
        
        #print(logits, type(logits))
        
        #logits_drop_ignore = [lg for (lg, label) in zip(logits, elem["labels"]) if label != -100]
        
        #print(logits_drop_ignore, type(logits_drop_ignore))
        
        #print(logits, logits.shape)
        probs = torch.nn.functional.softmax(logits, dim=-1)
        
        #print(probs, probs.shape, type(probs))
        ps = probs.squeeze()
        #print(ps, ps.shape)
        
        #[lg for (lg, label) in zip(logits, elem["labels"]) if label != -100]
        
        # SKIP -100 TOKENS
        np_ps = np.array([arr for arr, label in zip(ps.numpy(), elem["labels"]) if label != -100])
        
        #print(np_ps, np_ps.shape, type(np_ps))
        
        if not len(elem["tokens"]) == len(elem["ner_tags"]) == len(np_ps):
            print("LENGTH MISMATCH")
            print(elem)
            print(np_ps)
            break
        xtokens.append(elem["tokens"])
        xlabels.append(elem["ner_tags"])
        xpred_probs.append(np_ps)
        
        cnt +=1
        
        if cnt > 500:
            break

In [110]:
print(len(xtokens))
print(len(xlabels))
print(len(xpred_probs))

501
501
501


In [75]:
!pip install cleanlab

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.3/352.3 kB 7.8 MB/s eta 0:00:00:00:01


In [111]:
from cleanlab.token_classification.filter import find_label_issues
from cleanlab.token_classification.rank import get_label_quality_scores, issues_from_scores
from cleanlab.internal.token_classification_utils import get_sentence, filter_sentence, mapping
from cleanlab.token_classification.summary import display_issues, common_label_issues, filter_by_token

In [112]:
print(len(xlabels))
print(len(xpred_probs))

501
501


In [114]:
print(xtokens[113])

['Cairns', '8-1-35-1', '(', 'w-2', ')', ',', 'Vaughan', '9-1-55-1', ',', 'Harris', '10-0-42-5', '(', 'w-1', ')', ',']


In [115]:
print(xlabels[113])

[1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]


In [117]:
print(xpred_probs[113], len(xpred_probs[113]))

[[8.33258964e-04 9.82141733e-01 2.44669965e-03 6.23309519e-03
  5.03092946e-04 6.44248305e-03 3.42095824e-04 8.28961493e-04
  2.28598379e-04]
 [9.99816239e-01 4.01762300e-05 2.16481258e-05 1.59705942e-05
  1.94483473e-05 2.43368322e-05 1.73435728e-05 2.83760874e-05
  1.65951860e-05]
 [9.99807656e-01 5.31329606e-05 1.85042827e-05 2.00260365e-05
  1.58576640e-05 2.45612046e-05 1.40167112e-05 3.13046839e-05
  1.49245971e-05]
 [9.70011950e-01 2.12945901e-02 5.37827262e-04 6.44446176e-04
  1.44863065e-04 9.09121649e-04 2.22829593e-04 5.96641470e-03
  2.67969124e-04]
 [9.99816358e-01 4.45445949e-05 2.03619165e-05 1.71336542e-05
  1.86277302e-05 2.69581324e-05 1.73426652e-05 2.45389183e-05
  1.41739874e-05]
 [9.99765575e-01 8.95738704e-05 2.02524916e-05 2.27707933e-05
  1.76527828e-05 2.98566338e-05 1.42505905e-05 2.81451921e-05
  1.19086080e-05]
 [4.30384476e-04 9.94049668e-01 1.17266364e-03 2.09591887e-03
  2.38053821e-04 1.25851110e-03 2.10352606e-04 3.85437423e-04
  1.59043600e-04]
 [9.99

In [119]:
issues = find_label_issues(xlabels, xpred_probs)

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [125]:
label_list = conll["train"].features[f"ner_tags"].feature.names

label_list

#  DO THESE HAVE TO BE IN SOME ORDER FOR CLEANLAB??? SO UNCLEAR WHICH ORDER THEY MUST BE IN
#xclass_names = ['O','PER','ORG','LOC','MISC']
xclass_names = label_list

In [126]:
display_issues(issues,
               xtokens,
               pred_probs=xpred_probs,
               labels=xlabels,
              class_names=xclass_names)

Sentence index: 142, Token index: 2
Token: BLINKER
Given label: B-PER, predicted label according to provided pred_probs: O
----
SOCCER - BLINKER BAN LIFTED.


Sentence index: 356, Token index: 0
Token: League
Given label: B-ORG, predicted label according to provided pred_probs: O
----
League teams after games played on Thursday (tabulate under


Sentence index: 408, Token index: 0
Token: League
Given label: B-ORG, predicted label according to provided pred_probs: O
----
League games on Thursday (home team in CAPS):


Sentence index: 218, Token index: 12
Token: League
Given label: I-MISC, predicted label according to provided pred_probs: O
----
League title-holders Steaua Bucharest, who finished bottom of their Champions' League group in the European Cup, have only two players in the squad.


Sentence index: 200, Token index: 27
Token: Black
Given label: I-ORG, predicted label according to provided pred_probs: O
----
League duties restricted the Barbarians' selectorial options but they 

# Looking to see if e.g. the B-PER error with China is indeed in dataset

Seems pretty bad annotator error if so:

In [128]:
model.config.id2label

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

In [130]:
print(tokenized_conll["test"][0])
print("===")

for word, given_label in zip(tokenized_conll["test"][0]["tokens"], tokenized_conll["test"][0]["ner_tags"]):
    print(f"FOR ACTUAL WORD : {word} THE DATASET LABEL IS {model.config.id2label[given_label]}")

{'id': '0', 'tokens': ['SOCCER', '-', 'JAPAN', 'GET', 'LUCKY', 'WIN', ',', 'CHINA', 'IN', 'SURPRISE', 'DEFEAT', '.'], 'pos_tags': [21, 8, 22, 37, 22, 22, 6, 22, 15, 12, 21, 7], 'chunk_tags': [11, 0, 11, 21, 11, 12, 0, 11, 13, 11, 12, 0], 'ner_tags': [0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0], 'input_ids': [101, 4715, 1011, 2900, 2131, 5341, 2663, 1010, 2859, 1999, 4474, 4154, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0, -100]}
===
FOR ACTUAL WORD : SOCCER THE DATASET LABEL IS O
FOR ACTUAL WORD : - THE DATASET LABEL IS O
FOR ACTUAL WORD : JAPAN THE DATASET LABEL IS B-LOC
FOR ACTUAL WORD : GET THE DATASET LABEL IS O
FOR ACTUAL WORD : LUCKY THE DATASET LABEL IS O
FOR ACTUAL WORD : WIN THE DATASET LABEL IS O
FOR ACTUAL WORD : , THE DATASET LABEL IS O
FOR ACTUAL WORD : CHINA THE DATASET LABEL IS B-PER
FOR ACTUAL WORD : IN THE DATASET LABEL IS O
FOR ACTUAL WORD : SURPRISE THE DATASET LABEL IS O
FOR ACTUAL WORD : DEFE